In [36]:
    import pandas as pd, numpy as np, folium, re, json
from folium.plugins import AntPath

In [46]:
df = pd.read_csv("C:/Users/user00/Downloads/Yugoslav War Data.csv")

In [49]:
# CLEAN COLUMN NAMES
df.columns = df.columns.str.strip().str.replace(" ", "_")

# PARSE DISPLACEMENT RANGES TO MIDPOINT INTEGER
def parse_displacement(val):
    if isinstance(val, str) and '–' in val:
        nums = [int(x.replace(',', '')) for x in val.split('–')]
        return sum(nums) // 2
    elif isinstance(val, str):
        return int(val.replace(',', ''))
    return val

df["Number_Displaced"] = df["Number_Displaced"].apply(parse_displacement)

# MANUAL COORDINATES FOR ORIGINS/DESTINATIONS
country_coords = {
    "Croatia": [45.1, 15.2],
    "Bosnia and Herzegovina": [44.2, 17.7],
    "Germany": [51.2, 10.5],
    "Serbia": [44.0, 21.0],
    "Montenegro": [42.7, 19.4],
    "Albania": [41.3, 20.2],
    "North Macedonia": [41.6, 21.7],
    "Kosovo": [42.6, 20.9],
    "Other former Yugoslav republics": [44.0, 20.0],
    "Serbia and Montenegro": [43.8, 20.4],
    "Croatia and Bosnia": [44.7, 16.7],
    "Croatia (internal)": [45.1, 15.2],
    "Bosnia and Herzegovina (internal)": [44.2, 17.7],
    "Kosovo (internal)": [42.6, 20.9],
    "Serbia and Montenegro (internal)": [43.8, 20.4]
}

yugoslav_republics = [
    "Croatia", "Slovenia", "Bosnia and Herzegovina",
    "Republic of Serbia", "Montenegro", "North Macedonia", "Kosovo"
]

# LOAD EUROPEAN BORDERS GEOJSON
with open("1custom.geo.json", "r", encoding="utf-8") as f:
    europe_borders = json.load(f)

style_function = lambda feature: {
    'fillOpacity': 0.3 if feature['properties']['admin'] in yugoslav_republics else 0,
    'fillColor': '#555555' if feature['properties']['admin'] in yugoslav_republics else 'transparent',
    'color': 'black' if feature['properties']['admin'] in yugoslav_republics else 'gray',
    'weight': 2 if feature['properties']['admin'] in yugoslav_republics else 1
}

# LEGEND HTML
legend_html = """
<div style="
    position: fixed;
    bottom: 20px;
    left: 20px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid gray;
    font-size: 14px;
">
<b>Legend</b><br>
<span style="color:red;">&#9679;</span> Origin<br>
<span style="color:green;">&#9679;</span> Destination<br>
<span style="color:orange;">&#9679;</span> Internal displacement<br>
<span style="color:blue;">&#8594;</span> Refugee flow<br>
</div>
"""

# PERIOD TITLES & DESCRIPTIONS
period_titles = {
    "1991–1992": "Displacement During the Croatian War (1991–1992)",
    "1992–1995": "Displacement During the Bosnian War (1992–1995)",
    "1998–1999": "Displacement During the Kosovo War (1998–1999)"
}

period_descriptions = {
    "1991–1992": (
        "This map visualizes refugee and internal displacement patterns during the Croatian War. "
        "Large population movements occurred due to ethnic violence following Croatia’s declaration of independence."
    ),
    "1992–1995": (
        "This map captures the mass displacements of the Bosnian War, which involved ethnic cleansing, "
        "sieges, and forced migration across the Balkans. Bosnia and Herzegovina saw the largest internal displacements."
    ),
    "1998–1999": (
        "This map shows displacement from the Kosovo War, where ethnic Albanians were expelled en masse by Serbian forces. "
        "Refugees fled into neighboring Albania, Macedonia, and beyond."
    )
}

# LOOP OVER PERIODS
for period in df["Period"].unique():
    data = df[df["Period"] == period]
    m = folium.Map(location=[44.0, 20.0], zoom_start=5)

    folium.GeoJson(europe_borders, name="European Borders", style_function=style_function).add_to(m)

    # SUM TOTAL EXTERNAL DISPLACEMENTS BY ORIGIN
    ext_disp = data[~data["Destination_Country"].str.contains("internal", case=False)]
    origin_totals = ext_disp.groupby("Origin_Country")["Number_Displaced"].sum().to_dict()

    for _, row in data.iterrows():
        origin = row["Origin_Country"]
        destination = row["Destination_Country"]
        displaced = row["Number_Displaced"]
        dtype = row["Type"]

        if origin not in country_coords:
            continue

        origin_coords = country_coords[origin]
        origin_base = origin.replace(" (internal)", "")
        destination_base = destination.replace(" (internal)", "")
        is_internal = "internal" in destination.lower()

        lat_shift = 0.1
        lon_shift = 0.1
        origin_offset = [origin_coords[0] - lat_shift, origin_coords[1] - lon_shift]

        # HANDLE OTHER FORMER YUGOSLAV REPUBLICS
        if destination == "Other former Yugoslav republics":
            valid_dests = [r for r in yugoslav_republics if r != origin_base and r in country_coords]
            if not valid_dests:
                continue
            split = displaced // len(valid_dests)
            for dest in valid_dests:
                dest_coords = country_coords[dest]
                popup = f"{split:,} {dtype}<br>From: {origin}<br>To: {dest}"
                AntPath([origin_offset, dest_coords], color="blue", weight=2).add_to(m)
                folium.CircleMarker(dest_coords, radius=min(25, max(5, split / 50000)),
                                    color='green', fill=True, fill_opacity=0.8, popup=popup).add_to(m)
            continue

        # HANDLE SERBIA & MONTENEGRO SPLIT
        if destination in ["Serbia and Montenegro", "Serbia and Montenegro (internal)"]:
            split_val = displaced // 2
            target_suffix = " (internal)" if is_internal else ""
            for i, split_dest in enumerate(["Serbia", "Montenegro"]):
                if split_dest not in country_coords:
                    continue
                dest_coords = country_coords[split_dest]
                offset = [0.07 * (1 if i == 0 else -1), 0.07 * (1 if i == 0 else -1)]
                dest_coords_offset = [dest_coords[0] + offset[0], dest_coords[1] + offset[1]]
                popup = f"{split_val:,} {dtype}<br>To: {split_dest}{target_suffix}"

                if is_internal:
                    folium.CircleMarker(dest_coords_offset, radius=min(25, max(5, split_val / 50000)),
                                        color='orange', fill=True, fill_opacity=0.8, popup=popup).add_to(m)
                else:
                    AntPath([origin_offset, dest_coords], color="blue", weight=2).add_to(m)
                    folium.CircleMarker(dest_coords, radius=min(25, max(5, split_val / 50000)),
                                        color='green', fill=True, fill_opacity=0.8, popup=popup).add_to(m)
            continue

        if destination not in country_coords:
            continue

        destination_coords = country_coords[destination]
        popup = f"{displaced:,} {dtype}<br>From: {origin}<br>To: {destination}"

        if is_internal:
            offset_coords = [destination_coords[0] + lat_shift, destination_coords[1] + lon_shift]
            folium.CircleMarker(offset_coords, radius=min(25, max(5, displaced / 50000)),
                                color='orange', fill=True, fill_opacity=0.8, popup=popup).add_to(m)
        else:
            AntPath([origin_offset, destination_coords], color="blue", weight=max(2, displaced / 200000)).add_to(m)
            folium.CircleMarker(destination_coords, radius=min(25, max(5, displaced / 50000)),
                                color='green', fill=True, fill_opacity=0.8, popup=popup).add_to(m)

    # RED ORIGIN DOT SIZED BY TOTAL EXTERNAL DISPLACED
    for origin, total in origin_totals.items():
        if origin not in country_coords:
            continue
        coord = [country_coords[origin][0] - lat_shift, country_coords[origin][1] - lon_shift]
        folium.CircleMarker(coord, radius=min(25, max(5, total / 50000)),
                            color='red', fill=True, fill_opacity=0.8,
                            popup=f"Total externally displaced from {origin}: {total:,}").add_to(m)

    # ADD LEGEND, TITLE, SIDEBAR
    folium.LayerControl().add_to(m)

    # >>>> TITLE
    title_text = period_titles.get(period, f"Displacement Map: {period}")
    title_html = f"""
    <h3 style='position: fixed; top: 10px; left: 50%; transform: translateX(-50%);
         z-index:9999; background-color: white; padding: 10px; border: 2px solid gray;
         font-size: 20px;'>
    {title_text}
    </h3>
    """
    m.get_root().html.add_child(folium.Element(title_html))

    # >>>> SIDEBAR
    desc_text = period_descriptions.get(period, "")
    sidebar_html = f"""
    <div style="
        position: fixed;
        top: 80px;
        right: 20px;
        width: 300px;
        max-height: 500px;
        overflow-y: auto;
        z-index: 9999;
        background-color: white;
        padding: 15px;
        border: 2px solid gray;
        font-size: 14px;
    ">
    <h4>Description</h4>
    <p>{desc_text}</p>
    <ul>
        <li><b>Red dots</b>: Origin (total externally displaced)</li>
        <li><b>Green dots</b>: Destination</li>
        <li><b>Orange dots</b>: Internal displacement</li>
        <li><b>Blue lines</b>: Flow direction</li>
    </ul>
    </div>
    """
    m.get_root().html.add_child(folium.Element(sidebar_html))

    m.get_root().html.add_child(folium.Element(legend_html))

    # SAVE MAP
    safe_period = period.replace("–", "_").replace(" ", "")
    m.save(f"displacement_map_{safe_period}.html")
    print(f"Saved: displacement_map_{safe_period}.html")


Saved: displacement_map_1991_1992.html
Saved: displacement_map_1992_1995.html
Saved: displacement_map_1998_1999.html
